In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain

In [ ]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
CHAT_HISTORY_INDICATOR = 'chat_history_indicator'
HUMAN_INPUT = 'human_input'
CONTEXT = 'context'
SUPPORT_DOC_FOLDER_PATH = 'support_docs'

In [ ]:
def process_doc_files(SUPPORT_DOC_FOLDER_PATH) -> dict:
    """
    Process text files in a folder and return a dictionary with file names as keys and content as values.

    :param folder_path: The path to the folder containing the text files.
    :return: A dictionary with file names as keys and content as values.
    """
    # Initialize an empty dictionary to store the results
    doc_dict = {}

    # Check if the folder exists
    if not os.path.exists(SUPPORT_DOC_FOLDER_PATH):
        return doc_dict  # Return an empty dictionary if the folder does not exist

    # List all files in the folder
    file_list = os.listdir(SUPPORT_DOC_FOLDER_PATH)

    # Iterate through the files
    for filename in file_list:
        # Check if the file has a .txt extension
        if filename.endswith(".txt"):
            # Create the full path to the file
            file_path = os.path.join(SUPPORT_DOC_FOLDER_PATH, filename)

            # Open the file and read its content
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()

            # Store the content in the dictionary with the filename as the key
            doc_dict[filename] = file_content

    return doc_dict

In [ ]:
script_chain = None

In [ ]:
def backend_code(template_text: str, user_query: str  ):

    global script_chain

    embeddings = OpenAIEmbeddings()
    all_docs = list(process_doc_files(SUPPORT_DOC_FOLDER_PATH).values())
    docsearch = Chroma.from_texts(all_docs, embeddings)
    
    similar_docs = docsearch.similarity_search(user_query, k = 1)

    # initialize script chain if it doesn't exist
    if script_chain is None:
        llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0)
        prompt = PromptTemplate(input_variables = [CHAT_HISTORY_INDICATOR,HUMAN_INPUT, CONTEXT ], template= template_text )
        memory = ConversationBufferMemory(memory_key = CHAT_HISTORY_INDICATOR, input_key = HUMAN_INPUT)
        
        script_chain = load_qa_chain(llm = llm, chain_type = 'stuff', memory = memory, prompt = prompt)

    gen_ai_output = script_chain({"input_documents":similar_docs, HUMAN_INPUT: user_query}, return_only_outputs = True)

    print(script_chain.memory.buffer)

    return gen_ai_output['output_text']